In [46]:
from datetime import datetime
from zoneinfo import ZoneInfo
from typing import List
from dataclasses import dataclass
from os.path import join
import numpy as np
import pandas as pd
from hdmf.backends.hdf5 import H5DataIO
from pynwb import NWBFile, NWBHDF5IO, TimeSeries
from pynwb.file import Subject
from pynwb.ecephys import ElectricalSeries, ElectrodeGroup, LFP
from pynwb.behavior import BehavioralEvents
import nixio
import regex as re
from nwbwidgets import nwb2widget

In [47]:
@dataclass(frozen=True)
class SessionContext:
    subject: int
    session: int
    nix: nixio.File
    nwb: NWBFile

In [48]:
def convert_nix_to_nwb(subject: int, session: int) -> SessionContext:
    ctx = create_context(subject, session)
    write_subject(ctx)
    write_eeg_electrodes(ctx)
    write_eeg_measurements(ctx)
    ieeg_electrode_group = write_ieeg_electrodes(ctx)
    write_ieeg_measurements(ctx)
    write_trial_data(ctx)
    write_behavior(ctx)
    write_events(ctx)
    write_waveforms(ctx, ieeg_electrode_group)
    return ctx

In [49]:
def create_context(subject: int, session: int) -> SessionContext:
    nix = _read_nix(subject, session)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description=_get_task(nix),
        identifier=f"Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM_subject{subject:02}_session{session:02}",
        session_start_time=_get_date(subject, session),
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        related_publications=_get_related_publications(nix),
        experimenter="Boran, Ece",  # TODO is this right?
        experiment_description=_get_task(nix),  # Todo can we do better?
        keywords=["EEG", "iEEG", "LFP", "Epilepsy", "Memory", "Sternberg", "Verbal"],
    )
    return SessionContext(subject, session, nix, nwb)

In [50]:
def _read_nix(subject: int, session: int) -> nixio.File:
    _nix_dir = join(
        "/",
        "mnt",
        "c",
        "Users",
        "conta",
        "git",
        "USZ_NCH",
        "Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM",
        "data_nix",
    )
    _file_path = join(_nix_dir, f"Data_Subject_{subject:02}_Session_{session:02}.h5")
    return nixio.File.open(_file_path, nixio.FileMode.ReadOnly)

In [51]:
def _get_task(nix: nixio.File) -> str:
    task = nix.sections["Task"].props
    task_name = task["Task name"].values[0]
    # Broken UTF-8 in file
    task_desc = "The task is a modified Sternberg task in which the encoding of memory items, maintenance, and recall were temporally separated. Each trial starts with a fixation period ([-6, -5] s), followed by the stimulus ([-5, -3] s). The stimulus consists of a set of eight consonants at the center of the screen. The middle four, six, or eight letters are the memory items,which determine the set size for the trial (4, 6, or 8, respectively). The outer positions are filled with “X,” which is never a memory item. After the stimulus, the letters disappear from the screen, and the maintenance interval starts ([-3, 0] s).A fixation square is shown throughout fixation, encoding, and maintenance. After maintenance, a probe letter is presented. The subjects respond with a button press to indicate whether the probe was part of the stimulus.The subjects are instructed to respond as rapidly as possible without making errors. The hand used for the response is counterbalanced across subjects within the clinical constraints. After the response, the probe is turned off, and the subjects receive acoustic feedback regarding whether their response was correct or incorrect. Before initiating the next trial, the subjects were encouraged to blink and relax. The subjects perform 50 trials in one session, which last approximately 10 min. Trials with different set sizes are presented in a random order,with the single exception that a trial with an incorrect response is always followed by a trial with a set size of 4."
    task_url = task["Task URL"].values[0]
    return (
        f"Task Name: {task_name}\nTask Description: {task_desc}\nTask URL: {task_url}"
    )

In [52]:
def _get_date(subject: int, session: int) -> datetime:
    metadata = _get_metadata_row(subject, session)
    date = metadata["Date"].to_pydatetime()
    # IANA time zone format: https://en.wikipedia.org/wiki/List_of_tz_database_time_zones#ZURICH
    zone_info = ZoneInfo("Europe/Zurich")
    return date.replace(tzinfo=zone_info)

In [53]:
def _get_metadata_row(subject: int, session: int) -> pd.Series:
    df = _read_session_metadata()
    right_participant = df["Participant"] == subject
    right_session = df["Session"] == session
    right_dataset = df["Dataset"] == "Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM"
    return df.loc[right_participant & right_session & right_dataset].iloc[0]

In [54]:
def _read_session_metadata() -> pd.DataFrame:
    # This file was manually verified. Prefer the data in it over the one in the NIX file.
    trial_data_path = join("in", "metadata", "participants.csv")
    df = pd.read_csv(
        trial_data_path,
        usecols=[
            "Participant",
            "Session",
            "Age",
            "Sex",
            "Pathology",
            "sEEG electrodes analysed",
            "Electrodes in seizure onset zone (SOZ)",
            "Handedness",
            "Date",
            "Session Start Time",
            "Dataset"
        ],
        dtype={
            "Participant": np.int32,
            "Session": np.int32,
            "Age": np.int32,
            "Sex": str,
            "Pathology": str,
            "sEEG electrodes analysed": str,
            "Electrodes in seizure onset zone (SOZ)": str,
            "Handedness": str,
            "Date": str,
            "Session Start Time": str,
            "Dataset": str,
        },
    )
    df["Date"] = df["Date"] + " " + df["Session Start Time"]
    df["Date"] = df["Date"].apply(lambda date: datetime.strptime(date, "%y%m%d %H:%M"))
    df.drop(columns=["Session Start Time"], inplace=True)
    return df


In [55]:
def _get_related_publications(nix: nixio.File) -> List[str]:
    related_publications = (
        nix.sections["General"].sections["Related publications"].props
    )
    dois = related_publications["Publication DOI"].values
    return [doi.strip() for doi in dois]

In [56]:
def write_subject(ctx: SessionContext):
    metadata = _get_metadata_row(ctx.subject, ctx.session)
    age = metadata["Age"]
    sex = metadata["Sex"]
    ctx.nwb.subject = Subject(
        subject_id=f"{ctx.subject:02}",
        age=f"P{int(age)}Y",
        description=_get_subject_description(ctx),
        species="Homo sapiens",
        sex=_get_sex(sex),
    )

In [57]:
def _get_sex(raw: str) -> str:
    male = ["male", "m", "männlich", "maennlich", "mannlich", "männchen", "mannchen"]
    female = ["female", "f", "weiblich", "weibchen"]
    sex = raw.lower()
    return "M" if sex in male else "F" if sex in female else "O"

In [58]:
def _get_subject_description(ctx: SessionContext) -> str:
    metadata = _get_metadata_row(ctx.subject, ctx.session)
    subject = ctx.nix.sections["Subject"].props
    handedness = metadata["Handedness"]
    pathology = metadata["Pathology"]
    depth_electrodes = subject["Depth electrodes"].values[0]
    electrodes_in_soz = metadata["Electrodes in seizure onset zone (SOZ)"]
    return f"Handedness: {handedness}\nPathology: {pathology}\nDepth electrodes: {depth_electrodes}\nElectrodes in seizure onset zone (SOZ): {electrodes_in_soz}"

In [59]:
def write_eeg_electrodes(ctx: SessionContext):
    nwb = ctx.nwb

    device = nwb.create_device(
        name="NicoletOne EEG System",
        manufacturer="Natus Medical Incorporated",
        description="EEG recording system",
    )

    # create an electrode group for this group
    electrode_group = nwb.create_electrode_group(
        name=f"eeg",
        description=f"EEG electrodes on scalp",
        device=device,
        location="Scalp",
    )

    electrodes = _get_eeg_electrodes(ctx)
    electrodes.apply(
        lambda row: _add_row_to_eeg_electrodes(nwb, electrode_group, row), axis=1
    )

In [60]:
def _add_row_to_eeg_electrodes(
        nwb: NWBFile, electrode_group: ElectrodeGroup, row: pd.Series
):
    # got BESA map: +X is anterior, +Y is left, +Z is superior according to <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
    # But need NWB: +X is posterior, +Y is inferior, +Z is right according to <https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.add_electrode>
    nwb.add_electrode(
        group=electrode_group,
        location=row["label"],
        reference="Averaged mastoid channels",
        x=-row["x"],
        y=-row["z"],
        z=-row["y"],
        filtering="Passband, 0.5 to 5000 Hz",
    )

In [61]:
def _get_eeg_electrodes(ctx: SessionContext) -> pd.DataFrame:
    labels = _get_eeg_electrode_labels(ctx)
    locations = _get_eeg_electrode_locations(ctx)
    locations_array = np.ndarray(locations.shape)
    locations.read_direct(locations_array)
    df = pd.DataFrame(locations_array, columns=["x", "y", "z"])
    df.insert(0, "label", labels)
    return df.reset_index()

In [62]:
def _get_eeg_electrode_labels(ctx: SessionContext) -> List[str]:
    _assert_all_eeg_electrodes_have_same_labels(ctx)
    session_data = _get_session_data(ctx)
    return (
        session_data.groups["Scalp EEG data"]
        .data_arrays["Scalp_EEG_Data_Trial_01"]
        .dimensions[0]
        .labels
    )

In [63]:
def _assert_all_eeg_electrodes_have_same_labels(ctx: SessionContext):
    data_arrays = _get_session_data(ctx).groups["Scalp EEG data"].data_arrays
    electrode_labels = [data_array.dimensions[0].labels for data_array in data_arrays]
    assert len(set(electrode_labels)) == 1

In [64]:
def _get_session_data(ctx: SessionContext) -> nixio.Block:
    return ctx.nix.blocks[f"Data_Subject_{ctx.subject:02}_Session_{ctx.session:02}"]

In [65]:
def _get_eeg_electrode_locations(ctx: SessionContext) -> nixio.DataArray:
    return (
        _get_session_data(ctx)
        .groups["Scalp EEG electrode information"]
        .data_arrays["Scalp_Electrode_EEGLAB_BESA_Coordinates"]
    )

In [66]:
def write_ieeg_electrodes(ctx: SessionContext) -> ElectrodeGroup:
    nwb = ctx.nwb

    device = nwb.create_device(
        name="ATLAS Neurophysiology System",
        manufacturer="Neuralynx, Inc.",
        description="iEEG recording system",
    )

    # create an electrode group for this group
    electrode_group = nwb.create_electrode_group(
        name="ieeg",
        description=f"iEEG electrodes",
        device=device,
        location="Intracranial",
    )

    electrodes = _get_ieeg_electrodes(ctx)
    electrodes.apply(
        lambda row: _add_row_to_ieeg_electrodes(nwb, electrode_group, row), axis=1
    )
    return electrode_group

In [67]:
def _get_ieeg_electrodes(ctx: SessionContext) -> pd.DataFrame:
    labels = _get_ieeg_electrode_labels(ctx)
    locations = _get_ieeg_electrode_locations(ctx)
    locations_array = np.ndarray(locations.shape)
    locations.read_direct(locations_array)
    df = pd.DataFrame(locations_array, columns=["x", "y", "z"])
    df.insert(0, "label", labels)
    return df.reset_index()

In [68]:
def _get_ieeg_electrode_labels(ctx: SessionContext) -> List[str]:
    return (
        _get_session_data(ctx)
        .groups["iEEG electrode information"]
        .data_arrays["iEEG_Electrode_Manual_Entry"]
        .dimensions[0]
        .labels
    )

In [69]:
def _get_ieeg_electrode_locations(ctx: SessionContext) -> nixio.DataArray:
    return (
        _get_session_data(ctx)
        .groups["iEEG electrode information"]
        .data_arrays["iEEG_Electrode_MNI_Coordinates"]
    )

In [70]:
def _add_row_to_ieeg_electrodes(
        nwb: NWBFile, electrode_group: ElectrodeGroup, row: pd.Series
):
    # Got MNI map: +X is right, +Y is anterior, +Z is superior according to <https://kathleenhupfeld.com/mni-template-coordinate-systems/>
    # But need NWB: +X is posterior, +Y is inferior, +Z is right according to <https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.add_electrode>
    nwb.add_electrode(
        group=electrode_group,
        location=row["label"],
        reference="Common intracranial reference",
        x=-row["y"],
        y=-row["z"],
        z=row["x"],
        filtering="Passband, 0.5 to 1000 Hz",
    )

In [71]:
def write_eeg_measurements(ctx: SessionContext):
    nwb = ctx.nwb
    eeg_electrode_indices = list(range(_get_eeg_electrode_count(ctx)))
    eeg_table_region = nwb.create_electrode_table_region(
        region=eeg_electrode_indices,  # reference row indices 0 to N-1
        description="eeg electrodes",
    )
    trials = _get_session_data(ctx).groups["Scalp EEG data"].data_arrays
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    data = []
    timestamps = []
    for trial in trials:
        trial_number = int(_EEG_RE.findall(trial.name)[0])
        data.append(trial[:])
        sampling_interval = trial.dimensions[1].sampling_interval
        times = [
            i * sampling_interval - offset + trial_number * duration
            for i in range(trial.shape[1])
        ]
        timestamps.extend(times)
    data = np.concatenate(data, axis=1).transpose()
    raw_electrical_series = ElectricalSeries(
        name="ecephys.eeg",
        description="Scalp EEG data",
        data=data,
        electrodes=eeg_table_region,
        timestamps=timestamps,
    )
    nwb.add_acquisition(raw_electrical_series)

In [72]:
_EEG_RE = re.compile(r"Scalp_EEG_Data_Trial_(\d+)")

In [73]:
def _get_eeg_electrode_count(ctx: SessionContext) -> int:
    return len(_get_eeg_electrode_labels(ctx))

In [74]:
def write_ieeg_measurements(ctx: SessionContext):
    nwb = ctx.nwb
    min_index = _get_eeg_electrode_count(ctx)
    ieeg_electrode_indices = list(
        range(min_index, min_index + _get_ieeg_electrode_count(ctx))
    )
    ieeg_table_region = nwb.create_electrode_table_region(
        region=ieeg_electrode_indices,  # reference row indices 0 to N-1
        description="ieeg electrodes",
    )
    trials = _get_session_data(ctx).groups["iEEG data"].data_arrays
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    data = []
    timestamps = []
    for trial in trials:
        trial_number = int(_IEEG_RE.findall(trial.name)[0])
        data.append(trial[:])
        sampling_interval = trial.dimensions[1].sampling_interval
        times = [
            i * sampling_interval - offset + trial_number * duration
            for i in range(trial.shape[1])
        ]
        timestamps.extend(times)
    data = np.concatenate(data, axis=1).transpose()

    compressed_data = H5DataIO(
        data=data,
        compression="gzip",
    )
    electrical_series = ElectricalSeries(
        name="ecephys.ieeg",
        description="iEEG data",
        data=compressed_data,
        electrodes=ieeg_table_region,
        timestamps=timestamps,
    )
    lfp = LFP(electrical_series)
    ecephys_module = nwb.create_processing_module(
        name="ecephys", description="processed extracellular electrophysiology data"
    )
    ecephys_module.add(lfp)

In [75]:
_IEEG_RE = re.compile(r"iEEG_Data_Trial_(\d+)")

In [76]:
def _get_ieeg_electrode_count(ctx: SessionContext) -> int:
    return len(_get_ieeg_electrode_labels(ctx))

In [77]:
def write_events(ctx: SessionContext):
    nwb = ctx.nwb
    session = _get_session_data(ctx)
    tags = session.groups[
        "Trial events single tags spike times"
    ].tags  # same as EEG and iEEG in this case
    tags_by_trial = [(_EVENT_RE.findall(tag.name)[0], tag.position) for tag in tags]
    events = [
        (int(trial), name, position[0])
        for (name, trial), position in tags_by_trial
        if name != "Response"
    ]
    events.sort(key=lambda x: x[0])
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    events = [
        (name, time - offset + trial_number * duration)
        for trial_number, name, time in events
    ]
    events.append(("END", len(events) * duration))

    for (name, start), (_, end) in zip(events, events[1:]):
        nwb.add_epoch(
            start_time=start,
            stop_time=end,
            tags=name,
        )

In [78]:
def write_trial_data(ctx: SessionContext):
    nwb = ctx.nwb
    nwb.add_trial_column(name="trial_number", description="The trial number")
    nwb.add_trial_column(
        name="set_size",
        description="Number of letters shown during encoding period (4, 6, or 8 letters)",
    )
    nwb.add_trial_column(
        name="probe_letter",
        description="The letter presented to the participant during the retrieval period",
    )
    nwb.add_trial_column(
        name="response",
        description='The participant\'s answer to the question "Was the letter at hand present in the encoding set?"',
    )
    nwb.add_trial_column(
        name="response_time",
        description="The time at which the participant responded during the retrieval period",
    )
    nwb.add_trial_column(
        name="solution",
        description='The solution to the question "Was the letter at hand present in the encoding set?"',
    )
    nwb.add_trial_column(
        name="artifact",
        description="Whether the trial data was marked as an artifact by the experimenter",
    )

    nix = ctx.nix
    trials = nix.sections["Session"].sections["Trial properties"].sections
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    for trial in trials:
        trial = trial.props
        trial_number = int(trial["Trial number"].values[0])
        start_time = offset + trial_number * duration
        stop_time = start_time + duration
        response_time = trial["Response time"].values[0] + start_time
        # See https://gin.g-node.org/USZ_NCH/Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM/issues/2#issuecomment-3729
        response = str(trial["Response"].values[0])[1] == 1
        solution = int(trial["Match"].values[0]) == 1
        nwb.add_trial(
            start_time=start_time,
            stop_time=stop_time,
            trial_number=int(trial["Trial number"].values[0]),
            set_size=int(trial["Set size"].values[0]),
            probe_letter=str(trial["Probe letter"].values[0]),
            response=response,
            response_time=response_time,
            solution=solution,
            artifact=bool(trial["Artifact"].values[0]),
        )

In [79]:
def write_behavior(ctx: SessionContext):
    nwb = ctx.nwb
    behavior_module = nwb.create_processing_module(
        name="behavior", description="Data for all trials in this session."
    )
    nix = ctx.nix
    trials = nix.sections["Session"].sections["Trial properties"].sections
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    data = []
    timestamps = []
    for trial in trials:
        trial_number = int(trial["Trial number"])
        trial = trial.props
        # See https://gin.g-node.org/USZ_NCH/Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM/issues/2#issuecomment-3729
        data.append(str(trial["Response"].values[0])[1] == 1)
        time = trial["Response time"].values[0] - offset + trial_number * duration
        timestamps.append(time)
        # Have to do this because everything after the response is fake time inbetween trials because NWB requires all trials to be on one long stretch
        nwb.add_invalid_time_interval(
            start_time=time,
            stop_time=(trial_number + 1.0) * duration,
        )

    time_series = TimeSeries(
        name="response",
        data=data,
        timestamps=timestamps,
        description='The participant\'s answer to the question "Was the letter at hand present in the encoding set?"',
        unit="n/a",  # Might as well use https://github.com/rly/ndx-events, but it's not built-in...
        continuity="instantaneous",
    )

    behavioral_events = BehavioralEvents(
        name=f"BehavioralEvents.response", time_series=time_series
    )

    behavior_module.add(behavioral_events)

In [80]:
_EVENT_RE = re.compile(r"Event_([a-zA-Z]+)_.*Trial_(\d\d)_.*")

In [81]:
def write_waveforms(ctx: SessionContext, ieeg_electrode_group: ElectrodeGroup):
    nwb = ctx.nwb
    session = _get_session_data(ctx)
    waveforms = session.groups["Spike waveforms"].data_arrays
    spike_times = session.groups["Spike times"].data_arrays
    waveforms = [
        (_WAVEFORM_RE.findall(waveform.name)[0], waveform) for waveform in waveforms
    ]
    spike_times = [
        (_SPIKE_TIMES_RE.findall(spike_time.name)[0], spike_time[:])
        for spike_time in spike_times
    ]
    unit_to_waveform = {
        int(unit): (electrode, channel, values)
        for ((unit, electrode, channel), values) in waveforms
    }
    unit_to_trial_to_spike_times = {}
    for (unit, electrode, channel, trial), values in spike_times:
        unit_to_trial_to_spike_times.setdefault(int(unit), {})[trial] = (
            electrode,
            channel,
            values,
        )
    for unit, (electrode, channel, waveform) in unit_to_waveform.items():
        trial_to_spike_times = unit_to_trial_to_spike_times[unit]

        spike_times_for_trials = []
        for trial, (electrode_, channel_, spike_times) in trial_to_spike_times.items():
            assert electrode == electrode_
            assert channel == channel_
            spike_times_for_trials.append((trial, spike_times))
        spike_times_for_trials.sort(key=lambda x: x[0])
        spike_times_for_trials = [
            spike_times for _, spike_times in spike_times_for_trials
        ]
        spike_times_for_trials = _untrialize_irregular_timestamps(
            spike_times_for_trials, ctx
        )

        electrode_label = f"m{electrode}{channel}"
        electrode_index = _get_electrode_index(ctx, electrode_label)
        data_point_count = waveform.shape[1]
        sample_interval = waveform.dimensions[1].sampling_interval
        offset = waveform.dimensions[1].offset
        min_time = 0 + offset
        max_time = min_time + data_point_count * sample_interval
        data = waveform[:]
        nwb.add_unit(
            id=int(unit),
            electrode_group=ieeg_electrode_group,
            electrodes=[electrode_index],
            waveform_mean=data[0],
            waveform_sd=data[1],
            obs_intervals=[
                (time + min_time, time + max_time) for time in spike_times_for_trials
            ],
            spike_times=spike_times_for_trials,
        )

In [82]:
# Spike_Waveform_Unit_1_uAHL_2
_WAVEFORM_RE = re.compile(r"Spike_Waveform_Unit_(\d+)_u([a-zA-Z]+)_(\d+)")
# Spike_Times_Unit_36_uPHR_1_Trial_16
_SPIKE_TIMES_RE = re.compile(r"Spike_Times_Unit_(\d+)_u([a-zA-Z]+)_(\d+)_Trial_(\d+)")

In [83]:
def _untrialize_irregular_timestamps(
        timestamps: List[List[float]], ctx: SessionContext
) -> List[float]:
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    untrialized = []
    for trial, times in enumerate(timestamps):
        times = [time - offset + trial * duration for time in times]
        untrialized.extend(times)
    return untrialized

In [84]:
def _get_electrode_index(ctx: SessionContext, electrode: str) -> int:
    return ctx.nwb.electrodes["location"][:].index(electrode)

In [85]:
def _get_trial_duration(ctx: SessionContext) -> float:
    offset = _get_measurement_offset(ctx)
    return ctx.nix.sections["Session"].props["Trial duration"].values[0] - offset

In [86]:
def _get_measurement_offset(ctx: SessionContext) -> float:
    return (
        _get_session_data(ctx)
        .groups["Scalp EEG data"]
        .data_arrays[0]
        .dimensions[1]
        .offset
    )

In [87]:
def write_nwb(ctx: SessionContext):
    filename = f"subject{ctx.subject:02}_session{ctx.session:02}.nwb"
    with NWBHDF5IO(join("out", "converted", filename), "w") as io:
        io.write(ctx.nwb)

Main

In [88]:
context = convert_nix_to_nwb(1, 1)

In [89]:
write_nwb(context)

In [90]:
nwb2widget(context.nwb)

DELETE ANYTHING PYCHARM GENERATES AFTER THIS